**整数规划（Integer Programming，IP）**是**线性规划（Linear Programming，LP）**的一种扩展，其中**部分或全部决策变量被限制为整数**。

整数规划在很多实际问题中都能很好地应用，特别是在资源分配、调度问题、物流管理、网络设计等领域。由于整数规划的求解需要考虑整数约束，这使得其求解通常比普通的线性规划更加复杂。



## 1. 整数规划的标准形式

$$
\begin{aligned}
& \text{min/max} \quad c^T x \\
& ~~ \text{subject to}~~
\begin{cases}
Ax \leq b \\
x_i \in \mathbb{Z} \quad \text{for some or all} \quad i
\end{cases}
\end{aligned}
$$

其中：
- $ x \in \mathbb{R}^n $ 是决策变量向量，可能有些变量是整数，有些是连续变量。
- $ c $ 是目标函数的系数向量。
- $ A $ 是约束条件的系数矩阵，$ b $ 是约束条件的右侧向量。
- $ x_i \in \mathbb{Z} $ 表示第 $ i $ 个决策变量必须为整数，可能是所有变量（纯整数规划），也可能仅有部分变量（混合整数规划）。



* **0-1整数规划（0-1 Integer Programming）**

  0-1整数规划是一种特殊的整数规划，其**决策变量只能取两个值：0或1。**

  它常用于建模"是/否"类型的决策问题。例如：是否选择某个方案（1表示选择，0表示不选择）；是否建立某个设施（1表示建立，0表示不建立）。

  数学形式：

$$
\begin{aligned}
& \text{min/max}  ~~ c^T x \\
& ~~ \text{subject to}~~
\begin{cases}
Ax \leq b \\
x_i \in \{0, 1\} ~~ \text{for all} ~~ i
\end{cases}
\end{aligned}
$$


### 例1. 

利用`cvxpy`库求解下列整数线性规划问题
    $$
    \begin{aligned}
    & \min ~~ z=40x_1+90x_2  \\
    & ~~ \text{subject to} ~~ \left\{
    \begin{array}{l}
      9x_1+7x_2 \le 56, \\[.05in]
      7x_1+20x_2 \ge 70, \\[.05in]
      x_1,x_2\ge 0~\text{为整数}
    \end{array}
    \right.
    \end{aligned}
    $$

In [1]:
import cvxpy as cp

x1 = cp.Variable(integer=True)
x2 = cp.Variable(integer=True)

func = 40 * x1 + 90 * x2

objective = cp.Minimize(func)

constraints = [
    9*x1 + 7*x2 <= 56,
    7*x1 + 20*x2 >= 70,
    x1 >= 0,
    x2 >= 0
]

problem = cp.Problem(objective, constraints)   

problem.solve()

# 从定义的 Variable 变量里面获取解
x1_value = x1.value
x2_value = x2.value
objective_value = problem.value

x1_value, x2_value, objective_value


(array(2.), array(3.), np.float64(350.0))

### 例3

假设有 **5个物品**，每个物品的 **价值** 和 **重量** 如下表所示。现需要将这些物品放入一个容量为 **50kg** 的箱子中，目标是最大化装入箱子的总价值。每个物品只能选择装入或不装入。

| 物品 | 价值 $v_i$ (元) | 重量 $w_i$ (kg) |
|------|------------------------|------------------------|
| 1    | 60                     | 10                     |
| 2    | 100                    | 20                     |
| 3    | 120                    | 30                     |
| 4    | 80                     | 40                     |
| 5    | 50                     | 30                     |

箱子的容量为 **50**。

#### 数学建模

1. **决策变量**：
   - $ x_i \in \{0, 1\} $ 表示第 $i$ 个物品是否被装入箱子中。若 $x_i = 1$，则装入物品；若 $x_i = 0$，则不装入。

2. **目标函数**：
   - 我们的目标是最大化装入箱子的物品总价值，即：
     $$
     \max \sum_{i=1}^{5} v_i x_i
     $$
     其中 $v_i$ 是第 $i$ 个物品的价值。

3. **约束条件**：
   - **重量限制**：箱子的总重量不能超过容量 50 kg，即：
     $$
     \sum_{i=1}^{5} w_i x_i \leq 50
     $$
     其中 $w_i$ 是第 $i$ 个物品的重量。

4. **变量约束**：
   - $ x_i \in \{0, 1\} $，即每个物品只能选择装入或不装入箱子。

#### 标准形式


  $$
  \begin{aligned}
  & \max \sum_{i=1}^{5} v_i x_i \\
  & ~~\text{subject to} ~~
  \begin{cases}
  \displaystyle \sum_{i=1}^{5} w_i x_i \leq 50 \\
  x_i \in \{0, 1\} \quad \text{for} \quad i = 1, 2, \dots, 5
  \end{cases}
  \end{aligned}
  $$
  

In [ ]:
import cvxpy as cp
import numpy as np

values = np.array([60,100,120,80,50])

weights = np.array([10, 20, 30, 40, 50])

capacity = 50

# 提供一个 shape
x = cp.Variable(len(values), boolean=True)

objective = cp.Maximize(values @ x)

constraints = [
    weights @ x <= capacity,
    x >= 0
]

problem = cp.Problem(objective, constraints)

problem.solve()

x.value, problem.value



(array([0., 1., 1., 0., 0.]), np.float64(220.0))

### 综合建模

| 规格 | C$_1$ | C$_2$ | C$_3$ | C$_4$ | C$_5$ | C$_6$ | C$_7$ |
|------|--------|--------|--------|--------|--------|--------|--------|
| 厚度 $l$ (cm) | 48.7   | 52.0   | 61.3   | 72.0   | 48.7   | 52.0   | 64.0   |
| 重量 $w$ (kg) | 2000   | 3000   | 1000   | 500    | 4000   | 2000   | 1000   |
| 件数 $a$      | 8      | 7      | 9      | 6      | 6      | 4      | 8      |




假设我们有 **7种规格** 的包装箱，需要将它们装入 **两辆铁路平板车**，每辆平板车有 **10.2 米** 长的空间和 **40 吨** 的载重。为了使两辆平板车的空间和载重得到充分利用，目标是最优化装箱方案(求给出最好的装运方式，使得两辆铁路平板车的空间和载重得到充分利用。)，确保符合以下限制条件。

1. **决策变量**

设决策变量 $x_{ij}$ 表示在第 $i$ 辆平板车（$i = 1, 2$）上装第 $j$ 种包装箱（$j = 1, 2, \dots, 7$）的装载数量。

2. **约束条件**

    1. **空间限制**：
   每辆平板车的总厚度不能超过 10.2 米（即 1020 cm）。因此，平板车上的总厚度应满足：
   $$
   \sum_{j=1}^7 l_j \cdot x_{ij} \leq 1020 \quad \text{for} \quad i = 1, 2
   $$
   
    2. **载重限制**：
   每辆平板车的总重量不能超过 40 吨（即 40000 kg）。因此，平板车上的总重量应满足：
   $$
   \sum_{j=1}^7 w_j \cdot x_{ij} \leq 40000 \quad \text{for} \quad i = 1, 2
   $$
   其中，$w_i$ 是第 $i$ 种包装箱的重量（单位：千克）。

    3. **包装箱数量限制**：
   每种包装箱的数量不能超过给定的数量 $a_i$。即：
   $$
   0 \leq \sum_{i=1}^2 x_{ij} \leq a_j \quad \text{for} \quad j = 1, 2, \dots, 7
   $$
   
    4. **特殊限制**：
   对于 C$_5$、C$_6$、C$_7$ 类包装箱的总厚度有一个特别的限制，总厚度不能超过 302.7 cm。即：
   $$
   \sum_{j=5}^7 l_i \cdot (x_{1j}+x_{2j}) \leq 302.7 \quad \text{for} \quad i = 1, 2
   $$

ValueError: The 'maximize' objective must resolve to a scalar.

In [ ]:
# 厚度 L
L = np.array([48.7,52.0,61.3,72.0,48.7,52.0,64.0])
# 载重 w
w = np.array([2000,3000,1000,500,4000,2000,1000])
# 个数 a
a = np.array([8,7,9,6,6,4,8])

x = cp.Variable((2, len(L)), integer=True) # 两辆车

constraints = [
    
    x >= 0,
    x @ L <= 1020, 
    x @ w <= 40000,
    cp.sum(x[:, 4:] @ L[4 : ]) <= 302.7,
    # 件数要求
    # cp.sum(x,axis=0,keepdims=True)<=a.reshape(1,7),
    cp.sum(x, axis=0) <= a,
]

# 若希望装箱总厚度最大，即尽可能在体积上多装
objective = cp.Maximize(cp.sum(x @ L)) # 两个维度的和

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.GLPK_MI)
print(f"optimal value: {prob.value}")
print(f"optimal solution:\n{x.value}")


optimal value: 2039.4
optimal solution:
[[4. 1. 5. 3. 3. 2. 0.]
 [4. 6. 4. 3. 0. 1. 0.]]
